In [1]:
import numpy as np
import pandas as pd

In [8]:
path = 'M:/My Documents/Business/Property/Statements/processed/'

def load(spreadsheet, worksheet):
    #dir = 'E:/dtuklaptop/e/Users/Mat/python/data/property/'
    input_file = path + spreadsheet + '.xlsm'
    df = pd.read_excel(input_file,sheet_name=worksheet)
    df.drop('Import', axis=1, inplace=True)
    df.drop('Number', axis=1, inplace=True)
    df["Cat"] = np.nan
    df = df.astype({"Description": str})
    return df

def categorise(df):
    # Categorise expenses
    # Mortgage, PropertyExpense, OurRent, BealsRent, ServiceCharge
    # SchoolFees, Car, Hilltop, PersonalExpenses, RegularPayment
    global dfUncat
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('JASPER|TOPAZ|SIBERITE|MORTGAGE EXPRESS|NRAM|PLATFORM|AMBER|BHAM|CAPITAL|CHL|MORTGAGE TRUST')==True),"Cat"] = 'Mortgage'
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('.*M TUCKER.*STO.*')==True) & (df.Amount > -200) & (df.Amount <-190),"Cat"] = 'Mortgage' # Kingston Rd mortgage
    df.loc[(df.Cat.isnull())&(df.Description.notnull())&(df.Description.str.match('Rent for period')==True),"Cat"] = 'OurRent'
    df.loc[(df.Cat.isnull())&(df.Description.notnull())&(df.Property.notnull()) & (df.Description.str.match('Rent for period')==False),"Cat"] = 'PropertyExpense'
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('BEALS.*BGC')==True),"Cat"] = 'BealsRent'
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('23 HAMPSHIRE.*STO|4-6 ALHAMBRA RD CS|12-14 ALHAMBRA RD|16-18 ALHAMBRA RD|ALHAMBRA ROAD MANA')==True),"Cat"] = 'ServiceCharge'
    
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('.*HARPUR TRUST.*|.*BEDFORD SCHOOL.*')==True),"Cat"] = 'SchoolFees'
    
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('.*F VALENTINO.*')==True),"Cat"] = 'Hilltop'
    
    df.loc[(df.Cat.isnull())&(df.Subcategory=='PAYMENT'),"Cat"] = 'PersonalExpenses'
    df.loc[(df.Cat.isnull())&(df.Subcategory=='CASH'),'Cat'] = 'PersonalExpenses'
    df.loc[(df.Cat.isnull())&(df.Subcategory=='CRE'),'Cat'] = 'PersonalExpenses'
    df.loc[(df.Cat.isnull())&(df.Subcategory=='OTH'),'Cat'] = 'PersonalExpenses'
    df.loc[(df.Cat.isnull())&(df.Subcategory=='BCC'),'Cat'] = 'PersonalExpenses'
    df.loc[(df.Cat.isnull())&(df.Subcategory=='REFUND'),'Cat'] = 'PersonalExpenses'
    df.loc[(df.Cat.isnull())&(df.Subcategory=='CHQ'),'Cat'] = 'PersonalExpenses'
    
    df.loc[(df.Cat.isnull())&(df.Subcategory=='FT')&(df.Description.str.match('Alhambra',case=False)),'Cat'] = 'AlhambraPayment'
    
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('.*LAND ROVER.*')==True),"Cat"] = 'Car'
    df.loc[(df.Cat.isnull())&(df.Subcategory=='DIRECTDEBIT'),'Cat'] = 'RegularPayment'
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('NATIONWIDE|KINGSTON UNITY')==True),'Cat'] = 'RegularPayment'
    df.loc[(df.Cat.isnull())&(df.Subcategory=='REPEATPMT')&(df.Memo.str.match('.*M TUCKER.*STO.*')==True)&(df.Amount>-100),'Cat'] = 'RegularPayment'

    df.loc[(df.Cat.isnull())&(df.Subcategory=='FT')&(df.Memo.str.match('Sofia|Rocco', case=False)==True),'Cat'] = 'Kids'
    
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('.*30728691.*')==True),'Cat'] = 'Funds3072'
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('.*40406538 .*')==True),'Cat'] = 'Funds4040'
    df.loc[(df.Cat.isnull())&(df.Memo.str.match('.*60458872.*')==True),'Cat'] = 'Funds6045'
    #df.loc[(df.Cat.isnull()),'Cat'] = 'Uncategorised'
    
    # TODO: Add uncategorized to global dataframe, save to file
    dfUncat=dfUncat.append(df.loc[(df.Cat.isnull())])
    dfUncat = dfUncat.loc[:, ~dfUncat.columns.str.contains('^Unnamed')]
    dfUncat = dfUncat[~dfUncat.Account.isnull()]
    return df

def get_outgoings(dfOut,spreadsheet):
    input_file = path + spreadsheet + '.xlsm'
    dfRs = pd.read_excel(input_file,sheet_name='Rent Statement')
    stmtDate=dfRs.loc[dfRs['Rental Statement']=='Statement Date:','Unnamed: 1'].iloc[0]
        
    dfIv=load(spreadsheet, 'IV')
    dfIv=categorise(dfIv)
    
    dfMt=load(spreadsheet, 'MT')
    dfMt=categorise(dfMt)
    
    df40=load(spreadsheet, 'IV (4040)')
    df40=categorise(df40)

    totalMortgages = dfMt.loc[dfMt.Cat == "Mortgage","Amount"].sum() +dfIv.loc[dfIv.Cat == "Mortgage","Amount"].sum()
    totalPropertyExpense = dfMt.loc[dfMt.Cat == "PropertyExpense","Amount"].sum() +dfIv.loc[dfIv.Cat == "PropertyExpense","Amount"].sum()
    totalServiceCharge = dfMt.loc[dfMt.Cat == "ServiceCharge","Amount"].sum() +dfIv.loc[dfIv.Cat == "ServiceCharge","Amount"].sum()
    totalOurRent = dfMt.loc[dfMt.Cat == "OurRent","Amount"].sum() +dfIv.loc[dfIv.Cat == "OurRent","Amount"].sum()
    totalBealsRent = dfMt.loc[dfMt.Cat == "BealsRent","Amount"].sum() +dfIv.loc[dfIv.Cat == "BealsRent","Amount"].sum()
    
    totalHilltop = dfMt.loc[dfMt.Cat == "Hilltop","Amount"].sum() + \
                   dfIv.loc[dfIv.Cat == "Hilltop","Amount"].sum() + \
                   df40.loc[df40.Cat == "Hilltop","Amount"].sum()
    
    totalPersonalExpenses = dfMt.loc[dfMt.Cat == "PersonalExpenses","Amount"].sum() + \
                            dfIv.loc[dfIv.Cat == "PersonalExpenses","Amount"].sum() + \
                            df40.loc[df40.Cat == "PersonalExpenses","Amount"].sum()
    
    totalRegularPayment = dfMt.loc[dfMt.Cat == "RegularPayment","Amount"].sum() + \
                          dfIv.loc[dfIv.Cat == "RegularPayment","Amount"].sum() + \
                          df40.loc[df40.Cat == "RegularPayment","Amount"].sum()
    
    dfOut = dfOut.append({'Date': stmtDate, \
      'NetProfit': totalOurRent + totalBealsRent + totalMortgages + totalPropertyExpense + totalServiceCharge, \
      'SchoolFees': dfMt.loc[dfMt.Cat == "SchoolFees","Amount"].sum() + +dfIv.loc[dfIv.Cat == "SchoolFees","Amount"].sum(), \
      'Car': dfMt.loc[dfMt.Cat == "Car","Amount"].sum() + dfIv.loc[dfIv.Cat == "Car","Amount"].sum(), \
      'Hilltop': totalHilltop, \
      'PersonalExpenses': totalPersonalExpenses, \
      'RegularPayment': dfMt.loc[dfMt.Cat == "RegularPayment","Amount"].sum() + +dfIv.loc[dfIv.Cat == "RegularPayment","Amount"].sum() \
     }, \
                 ignore_index=True)
    dfOut['TotalOutgoings'] = dfOut['SchoolFees'] + dfOut['Car'] + dfOut['Hilltop'] + dfOut['PersonalExpenses'] + dfOut['RegularPayment']
    return dfOut

# add subcats for pie chart - food, home, fast food etc

In [9]:
spreadsheets=[ \
              'Beals_RentalStatement_JUN2019_08082019_2041', 'Beals_RentalStatement_JUL2019_12102019_2048', \
              'Beals_RentalStatement_AUG2019_12102019_2049', 'Beals_RentalStatement_SEP2019_01122019_2054', \
              'Beals_RentalStatement_OCT2019_05042020_2058', 'Beals_RentalStatement_NOV2019_05042020_2059', \
              'Beals_RentalStatement_DEC2019_05042020_2061', 'Beals_RentalStatement_JAN2020_05042020_2063', \
              'Beals_RentalStatement_FEB2020_05042020_2065', 'Beals_RentalStatement_MAR2020_20082020_2066', \
              'Beals_RentalStatement_APR2020_21082020_2071', 'Beals_RentalStatement_MAY2020_28082020_2082', \
              'Beals_RentalStatement_JUN2020_28082020_2083', 'Beals_RentalStatement_JUL2020_28082020_2084', \
              'Beals_RentalStatement_AUG2020_23062021_2098', 'Beals_RentalStatement_SEP2020_23062021_2105', \
              'Beals_RentalStatement_OCT2020_23062021_2109', 'Beals_RentalStatement_NOV2020_23062021_2124', \
              'Beals_RentalStatement_DEC2020_23062021_2125']

dfUncat = pd.DataFrame()
dfOut = pd.DataFrame(columns=['Date','NetProfit','TotalOutgoings','SchoolFees','Car','Hilltop','PersonalExpenses','RegularPayment'])
for spreadsheet in spreadsheets:
    dfOut=get_outgoings(dfOut, spreadsheet)
dfOut=dfOut.set_index('Date')

dfOut
# index by date

,NetProfit,TotalOutgoings,SchoolFees,Car,Hilltop,PersonalExpenses,RegularPayment
Date,,,,,,,
2019-06-30,-5071.76,-12009.60,-2637.40,-476.44,-102.0,-8112.42,-681.34
2019-07-31,-219.98,-9079.00,-4433.06,-476.44,-102.0,-3201.12,-866.38
2019-08-31,5626.45,-4577.49,0.00,-476.44,-902.0,-2453.78,-745.27
2019-09-30,5370.30,-13836.54,-10264.84,-476.44,-102.0,-2139.88,-853.38
2019-10-31,4139.24,-7772.43,-2381.50,-476.44,-102.0,-3940.62,-871.87
2019-11-30,2874.60,-5272.14,-1280.75,-476.44,-102.0,-2765.86,-647.09
2019-12-31,6000.45,-4371.41,0.00,-476.44,-302.0,-2848.44,-744.53
2020-01-31,8647.64,-15273.77,-11453.53,-476.44,-102.0,-2441.68,-800.12
2020-02-29,3879.85,-15995.13,-4624.22,-476.44,-102.0,-10141.02,-651.45


In [11]:
#dfUncat = dfUncat[~dfUncat.Account.isnull()]
dfUncat.head(20)

,Account,Amount,Cat,Date,Description,Memo,Property,Subcategory
0,20-74-09 60458872,559.25,NaN,2019-06-03,ALHAMBRA POSTBOXES,209254 13686035 POST BOX 16-18 FT,NaN,FT
1,20-74-09 60458872,559.25,NaN,2019-06-03,ALHAMBRA POSTBOXES,209254 23534332 POST BOXES 12-14 FT,NaN,FT
79,20-74-09 60458872,2950.00,NaN,2019-06-05,Mark Jones - 16-18 Alhambra Rd,209254 13686035 BLOCK 16-18 PAINT FT,NaN,FT
73,20-74-09 40406538,-20.00,NaN,2019-06-21,nan,CLAUDIA TOLLETT THANKS FT,NaN,FT
89,20-74-09 40406538,-10.00,NaN,2019-06-26,nan,L GREENWOOD ROCCO GIFT FT,NaN,FT
0,20-74-09 40406538,-33.00,NaN,2019-07-01,nan,TRACEY OWNERS LEO FT,NaN,FT
1,20-74-09 40406538,-25.00,NaN,2019-07-01,nan,KATIE GUEST ALESSIO FT,NaN,FT
2,20-74-09 40406538,-15.00,NaN,2019-07-01,nan,S BARNES ROCCO FT,NaN,FT
20,20-74-09 40406538,-5.00,NaN,2019-07-04,nan,L GREENWOOD THANK YOU LISA XX FT,NaN,FT
64,20-74-09 40406538,-20.00,NaN,2019-07-15,nan,CLAUDIA TOLLETT MARVELLOUS BOUFANT FT,NaN,FT
